# Proyecto Acrobot con NEAT


In [11]:
!pip install coppeliasim-zmqremoteapi-client

In [12]:
!pip install cbor

In [13]:
import numpy as np
import time

from python.zmqRemoteApi import RemoteAPIClient


class Coppelia():

    def __init__(self):
        print('*** connecting to coppeliasim')
        client = RemoteAPIClient()
        self.sim = client.getObject('sim')

    def start_simulation(self):
        # print('*** saving environment')
        self.default_idle_fps = self.sim.getInt32Param(self.sim.intparam_idle_fps)
        self.sim.setInt32Param(self.sim.intparam_idle_fps, 0)
        self.sim.startSimulation()

    def stop_simulation(self):
        # print('*** stopping simulation')
        self.sim.stopSimulation()
        while self.sim.getSimulationState() != self.sim.simulation_stopped:
            time.sleep(0.1)
        # print('*** restoring environment')
        self.sim.setInt32Param(self.sim.intparam_idle_fps, self.default_idle_fps)
        print('*** done')

    def is_running(self):
        return self.sim.getSimulationState() != self.sim.simulation_stopped


In [15]:
import sim
import numpy as np
import pandas as pd

In [87]:
class Acrobot():
    
    num_sonar = 16
    ang_max = np.pi
    joint_pos=0
    joint2_handle=-1
    dummy_handle=-1

    def __init__(self, sim, robot_id, use_camera=False, use_lidar=False):
        self.sim = sim
        print('*** getting handles', robot_id)
        #self.joint1 = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1')
        #self.joint2 = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1/Link1/Joint2')
        self.joint2_handle = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1/Link1/Joint2')
        self.dummy_handle = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1/Link1/Joint2/Link2/Dummy')


    def set_speed(self, speed):
        self.sim.setJointTargetVelocity(self.joint2_handle, speed)
    def set_pos_joint(self,pos):
        pos=pos*np.pi/180
        self.sim.setJointTargetPosition(self, pos,  [])
    def set_joint_torque(self,torque):
        self.sim.setJointTargetForce( self.joint2_handle, torque, True)
    def get_handle(self):
        return self.joint2_handle

    def get_pos_joint(self):
        self.joint_pos=self.sim.getJointTargetPosition(self.joint2_handle)*180/np.pi
        return self.joint_pos
    def get_pos_dummy(self):
        return self.sim.getObjectPosition(self.dummy_handle,self.sim.handle_world)
         
    def get_velocity_joint(self):
        return  self.sim.getJointTargetVelocity(self.joint2_handle)
    def get_torque_joint(self):
        return self.sim.getJointForce(self.joint2_handle)
    def start_motor(self):
        self.sim.setObjectInt32Param(self.joint2_handle,sim.sim_jointintparam_motor_enabled,1)
        #self.sim.setObjectInt32Param(self.joint2_handle,sim.sim_jointintparam_ctrl_enabled,1)


## Conexión

## Obtener Manejadores

In [106]:
def main(args=None):
    coppelia = Coppelia()
    robot = Acrobot(coppelia.sim, 'Cuboid')
    #robot.set_speed(+1.2)
    coppelia.start_simulation()
    robot.start_motor()
    #robot.set_speed(+3.14)
    robot.set_joint_torque(0.2)
    t1=0
    print(f'Position Dummy:',robot.get_pos_dummy()[2] )
    while (t := coppelia.sim.getSimulationTime()) < 5:
        print(f'Simulation time: {t:.3f} [s]')
        print(f'Velocity joint: ', robot.get_velocity_joint())
        print(f'Torque joint: ',robot.get_torque_joint())
        '''if t1%2==0:
            robot.set_speed(-3.14)
        else:
            robot.set_speed(3.14)'''
        print(f'Position Dummy:',robot.get_pos_dummy()[2] )
        if t1%2==0:
            robot.set_joint_torque(-0.2)
        else:
            robot.set_joint_torque(0.2)
        print(f' grados: {robot.get_pos_joint():.2f}')
        t1+=1

    coppelia.stop_simulation()

In [107]:
#+2.2275 altura máxima del dummy
#con torque 0.2 no llega a subir
#fitness será el que más tiempo esté el dummy en la posición más alta
#Exito: dejar x segundos el dummy en la posición más alta
#ctrl+w cierra escena en copelia
main()

*** connecting to coppeliasim
*** getting handles Cuboid
Position Dummy: -4.07556645490518
Simulation time: 0.050 [s]
Velocity joint:  1000.0
Torque joint:  -0.19999999999999998
Position Dummy: 1.863241998044734
 grados: 0.00
Simulation time: 0.300 [s]
Velocity joint:  -1000.0
Torque joint:  0.19999999999999998
Position Dummy: 1.4448751870652305
 grados: 0.00
Simulation time: 0.450 [s]
Velocity joint:  1000.0
Torque joint:  -0.19999999999999998
Position Dummy: 0.37420129860265217
 grados: 0.00
Simulation time: 0.650 [s]
Velocity joint:  -1000.0
Torque joint:  0.19999999999999998
Position Dummy: 0.7456121745742819
 grados: 0.00
Simulation time: 0.900 [s]
Velocity joint:  1000.0
Torque joint:  -0.19999999999999998
Position Dummy: 1.1673184092131086
 grados: 0.00
Simulation time: 1.100 [s]
Velocity joint:  -1000.0
Torque joint:  0.19999999999999998
Position Dummy: 1.2293126611274388
 grados: 0.00
Simulation time: 1.300 [s]
Velocity joint:  1000.0
Torque joint:  -0.19999999999999998
Positi